# NeuralNet Multi-Layer Perceptron

In [1]:
# Activation Function: ReLU
# Loss Function: Cross-Entropy Error

# Using for SoftmaxWithLoss.forward()
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# Using for SoftmaxWithLoss.forward()
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [2]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [3]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [4]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [5]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from collections import OrderedDict

# TwoLayer
class NeuralNetMLP:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # x: 입력 데이터, t: 정답 레이블
#     def numerical_gradient(self, x, t):
#         loss_W = lambda W: self.loss(x, t)
        
#         grads = {}
#         grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
#         grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
#         grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
#         grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
#         return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

In [6]:
from tqdm.auto import tqdm
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = NeuralNetMLP(input_size=784, hidden_size=512, output_size=10)

epochs = 100
idx_arr = np.arange(len(x_train))
batch_size = 100
learning_rate = 0.00003


for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list), train_acc*100, test_acc*100))

Epoch : 0 / Loss : 690.7653 / Train Acc : 9.1750% / Test Acc : 8.9500%
Epoch : 1 / Loss : 690.6791 / Train Acc : 10.9300% / Test Acc : 10.6800%
Epoch : 2 / Loss : 690.5928 / Train Acc : 12.9383% / Test Acc : 12.8000%
Epoch : 3 / Loss : 690.5063 / Train Acc : 15.3883% / Test Acc : 15.0100%
Epoch : 4 / Loss : 690.4197 / Train Acc : 18.1817% / Test Acc : 17.8100%
Epoch : 5 / Loss : 690.3328 / Train Acc : 21.0817% / Test Acc : 20.9400%
Epoch : 6 / Loss : 690.2454 / Train Acc : 23.6617% / Test Acc : 23.3400%
Epoch : 7 / Loss : 690.1576 / Train Acc : 26.0450% / Test Acc : 25.9200%
Epoch : 8 / Loss : 690.0691 / Train Acc : 28.0117% / Test Acc : 28.0600%
Epoch : 9 / Loss : 689.9800 / Train Acc : 29.6800% / Test Acc : 29.5100%
Epoch : 10 / Loss : 689.8902 / Train Acc : 30.9283% / Test Acc : 30.6700%
Epoch : 11 / Loss : 689.7995 / Train Acc : 31.9067% / Test Acc : 31.4700%
Epoch : 12 / Loss : 689.7080 / Train Acc : 32.6950% / Test Acc : 32.1100%
Epoch : 13 / Loss : 689.6157 / Train Acc : 33.2800

In [16]:
# ThreeLayer
class NeuralNetMLP:
    
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size1)
        self.params['b1'] = np.zeros(hidden_size1)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size1, hidden_size2)
        self.params['b2'] = np.zeros(hidden_size2)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size2, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        grads['W3'] = numerical_gradient(loss_W, self.params['W3'])
        grads['b3'] = numerical_gradient(loss_W, self.params['b3'])
        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine3'].dW
        grads['b3'] = self.layers['Affine3'].db
        
        return grads

In [17]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = NeuralNetMLP(input_size=784, hidden_size1=256, hidden_size2=256, output_size=10)

epochs = 100

idx_arr = np.arange(len(x_train))

batch_size = 1000
learning_rate = 0.00003

for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list),
                                                                                         train_acc*100, test_acc*100))

Epoch : 0 / Loss : 9209.2569 / Train Acc : 7.9700% / Test Acc : 8.1200%
Epoch : 1 / Loss : 9209.2557 / Train Acc : 7.9900% / Test Acc : 8.1500%
Epoch : 2 / Loss : 9209.2545 / Train Acc : 8.0667% / Test Acc : 8.2500%
Epoch : 3 / Loss : 9209.2533 / Train Acc : 8.1100% / Test Acc : 8.3200%
Epoch : 4 / Loss : 9209.2521 / Train Acc : 8.1667% / Test Acc : 8.4700%
Epoch : 5 / Loss : 9209.2509 / Train Acc : 8.2133% / Test Acc : 8.5600%
Epoch : 6 / Loss : 9209.2497 / Train Acc : 8.2883% / Test Acc : 8.7300%
Epoch : 7 / Loss : 9209.2485 / Train Acc : 8.3617% / Test Acc : 8.7900%
Epoch : 8 / Loss : 9209.2473 / Train Acc : 8.4317% / Test Acc : 8.8600%
Epoch : 9 / Loss : 9209.2461 / Train Acc : 8.5117% / Test Acc : 8.9600%
Epoch : 10 / Loss : 9209.2449 / Train Acc : 8.5967% / Test Acc : 9.0800%
Epoch : 11 / Loss : 9209.2437 / Train Acc : 8.6967% / Test Acc : 9.2000%
Epoch : 12 / Loss : 9209.2425 / Train Acc : 8.7833% / Test Acc : 9.3200%
Epoch : 13 / Loss : 9209.2413 / Train Acc : 8.8900% / Test Ac

In [19]:
# FiveLayer
class NeuralNetMLP:
    
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, hidden_size5, 
                 output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size1)
        self.params['b1'] = np.zeros(hidden_size1)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size1, hidden_size2)
        self.params['b2'] = np.zeros(hidden_size2)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size2, hidden_size3)
        self.params['b3'] = np.zeros(hidden_size3)
        self.params['W4'] = weight_init_std * np.random.randn(hidden_size3, hidden_size4)
        self.params['b4'] = np.zeros(hidden_size4)
        self.params['W5'] = weight_init_std * np.random.randn(hidden_size4, hidden_size5)
        self.params['b5'] = np.zeros(hidden_size5)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3'] = Affine(self.params['W3'], self.params['b3'])
        self.layers['Relu3'] = Relu()
        self.layers['Affine4'] = Affine(self.params['W4'], self.params['b4'])
        self.layers['Relu4'] = Relu()
        self.layers['Affine5'] = Affine(self.params['W5'], self.params['b5'])
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        grads['W3'] = numerical_gradient(loss_W, self.params['W3'])
        grads['b3'] = numerical_gradient(loss_W, self.params['b3'])
        grads['W4'] = numerical_gradient(loss_W, self.params['W4'])
        grads['b4'] = numerical_gradient(loss_W, self.params['b4'])
        grads['W5'] = numerical_gradient(loss_W, self.params['W5'])
        grads['b5'] = numerical_gradient(loss_W, self.params['b5'])

        return grads
    
    def gradient(self, x, t):
        self.loss(x, t)
        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine3'].dW
        grads['b3'] = self.layers['Affine3'].db
        grads['W4'] = self.layers['Affine4'].dW
        grads['b4'] = self.layers['Affine4'].db
        grads['W5'] = self.layers['Affine5'].dW
        grads['b5'] = self.layers['Affine5'].db
        
        return grads

In [20]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# load data
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = NeuralNetMLP(input_size=784, hidden_size1=256, hidden_size2=128, hidden_size3=98, hidden_size4=50, hidden_size5=10, 
                       output_size=10)

epochs = 100

idx_arr = np.arange(len(x_train))

batch_size = 1000
learning_rate = 0.00003

for epoch_i in range(epochs):
    train_loss_list = []
    np.random.shuffle(idx_arr)
    for idx in idx_arr.reshape(-1, batch_size):
        
        x_batch, t_batch = x_train[idx], t_train[idx]
        grad = network.gradient(x_batch, t_batch)
        
        for key in ('W1', 'b1', 'W2', 'b2', 'W3', 'b3', 'W4', 'b4', 'W5', 'b5'):
            network.params[key] -= learning_rate * grad[key]
            
        loss = network.loss(x_batch, t_batch)
        train_loss_list.append(loss)
    
    train_acc = network.accuracy(x_train, t_train)
    test_acc = network.accuracy(x_test, t_test)
    
    print('Epoch : {} / Loss : {:.4f} / Train Acc : {:.4f}% / Test Acc : {:.4f}%'.format(epoch_i, np.mean(train_loss_list),
                                                                                         train_acc*100, test_acc*100))

Epoch : 0 / Loss : 9209.3404 / Train Acc : 11.2483% / Test Acc : 11.3700%
Epoch : 1 / Loss : 9209.3399 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 2 / Loss : 9209.3393 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 3 / Loss : 9209.3388 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 4 / Loss : 9209.3383 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 5 / Loss : 9209.3377 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 6 / Loss : 9209.3372 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 7 / Loss : 9209.3367 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 8 / Loss : 9209.3361 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 9 / Loss : 9209.3356 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 10 / Loss : 9209.3351 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 11 / Loss : 9209.3346 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 12 / Loss : 9209.3341 / Train Acc : 11.2367% / Test Acc : 11.3500%
Epoch : 13 / Loss : 9209.3335 / Tra

KeyboardInterrupt: 

In [41]:
a = Abc()

In [42]:
a